In [45]:
import requests
import config
import mysql.connector 

In [40]:
parameters = {'api_key': config.api_key}

In [41]:
# response.json()

In [42]:
#genres table info
parameters = {'api_key': config.api_key}
response = requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=6b07ed055b275066e1c75fa784bba848&language=en-US')
response.status_code
response.json()






{'genres': [{'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'},
  {'id': 16, 'name': 'Animation'},
  {'id': 35, 'name': 'Comedy'},
  {'id': 80, 'name': 'Crime'},
  {'id': 99, 'name': 'Documentary'},
  {'id': 18, 'name': 'Drama'},
  {'id': 10751, 'name': 'Family'},
  {'id': 14, 'name': 'Fantasy'},
  {'id': 36, 'name': 'History'},
  {'id': 27, 'name': 'Horror'},
  {'id': 10402, 'name': 'Music'},
  {'id': 9648, 'name': 'Mystery'},
  {'id': 10749, 'name': 'Romance'},
  {'id': 878, 'name': 'Science Fiction'},
  {'id': 10770, 'name': 'TV Movie'},
  {'id': 53, 'name': 'Thriller'},
  {'id': 10752, 'name': 'War'},
  {'id': 37, 'name': 'Western'}]}

In [46]:
parameters = {'api_key': config.api_key}
response = requests.get('https://api.themoviedb.org/3/discover/movie?sort_by=revenue.desc', params= parameters)
response.status_code
response.json()

{'page': 1,
 'total_results': 428623,
 'total_pages': 21432,
 'results': [{'vote_count': 0,
   'id': 617589,
   'video': False,
   'vote_average': 0,
   'title': 'Death Is Just Around the Corner 2',
   'popularity': 0.876,
   'poster_path': None,
   'original_language': 'en',
   'original_title': 'Death Is Just Around the Corner 2',
   'genre_ids': [],
   'backdrop_path': None,
   'adult': False,
   'overview': 'Inspired by the masterpiece Pine 2',
   'release_date': ''},
  {'vote_count': 8050,
   'id': 299534,
   'video': False,
   'vote_average': 8.4,
   'title': 'Avengers: Endgame',
   'popularity': 234.9,
   'poster_path': '/or06FN3Dka5tukK1e9sl16pB3iy.jpg',
   'original_language': 'en',
   'original_title': 'Avengers: Endgame',
   'genre_ids': [12, 878, 28],
   'backdrop_path': '/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg',
   'adult': False,
   'overview': "After the devastating events of Avengers: Infinity War, the universe is in ruins due to the efforts of the Mad Titan, Thanos. With the 